<a href="https://colab.research.google.com/github/gmasihy/UC/blob/main/swissmetro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Swissmetro dataset
El conjunto de datos de swissmetro contiene preferencias declaradas para tres modos de transporte alternativos que incluyen automóvil, tren y un modo recientemente introducido: el swissmetro. 

Los datos pueden descargarse directamente desde Python utilizando el código presentado en la siguiente celda:

In [1]:
import pandas as pd
import numpy as np

df = pd.read_table("http://transp-or.epfl.ch/data/swissmetro.dat", sep='\t').dropna()
df

,GROUP,SURVEY,SP,ID,PURPOSE,FIRST,TICKET,WHO,LUGGAGE,AGE,...,TRAIN_TT,TRAIN_CO,TRAIN_HE,SM_TT,SM_CO,SM_HE,SM_SEATS,CAR_TT,CAR_CO,CHOICE
0,2,0,1,1,1,0,1,1,0,3,...,112,48,120,63,52,20,0,117,65,2
1,2,0,1,1,1,0,1,1,0,3,...,103,48,30,60,49,10,0,117,84,2
2,2,0,1,1,1,0,1,1,0,3,...,130,48,60,67,58,30,0,117,52,2
3,2,0,1,1,1,0,1,1,0,3,...,103,40,30,63,52,20,0,72,52,2
4,2,0,1,1,1,0,1,1,0,3,...,130,36,60,63,42,20,0,90,84,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10723,3,1,1,1192,4,1,7,1,0,5,...,148,13,30,93,17,30,0,156,56,2
10724,3,1,1,1192,4,1,7,1,0,5,...,148,12,30,96,16,10,0,96,70,3
10725,3,1,1,1192,4,1,7,1,0,5,...,148,16,60,93,16,20,0,96,56,3
10726,3,1,1,1192,4,1,7,1,0,5,...,178,16,30,96,17,30,0,96,91,2


In [2]:
df['GROUP'].value_counts()

3    6759
2    3969
Name: GROUP, dtype: int64

Las variables incluyen aspectos comunes para los tres modos alternativos, como el propósito del viaje (`PURPOSE`), y variables con valores específico para cada uno, que capturan tiempo de viaje (`TT`), el _headway_ (`HE`) y el costo (`CO`). La columna `CHOICE` indica el modo elegido por el encuestado. Más detalles pueden encontrarse [acá](http://www.sze.hu/~prile/Doktori%20iskola%202018/adatok%20leiras.pdf).

Con el fin de filtrar datos que no provean aspectos relevantes para el problema presentado en la tarea, es conveniente filtrar aquellas filas donde la elección de modo sea desconocida (`CHOICE` igual a 0).

In [3]:
df = df[df['CHOICE'] != 0]
df

,GROUP,SURVEY,SP,ID,PURPOSE,FIRST,TICKET,WHO,LUGGAGE,AGE,...,TRAIN_TT,TRAIN_CO,TRAIN_HE,SM_TT,SM_CO,SM_HE,SM_SEATS,CAR_TT,CAR_CO,CHOICE
0,2,0,1,1,1,0,1,1,0,3,...,112,48,120,63,52,20,0,117,65,2
1,2,0,1,1,1,0,1,1,0,3,...,103,48,30,60,49,10,0,117,84,2
2,2,0,1,1,1,0,1,1,0,3,...,130,48,60,67,58,30,0,117,52,2
3,2,0,1,1,1,0,1,1,0,3,...,103,40,30,63,52,20,0,72,52,2
4,2,0,1,1,1,0,1,1,0,3,...,130,36,60,63,42,20,0,90,84,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10723,3,1,1,1192,4,1,7,1,0,5,...,148,13,30,93,17,30,0,156,56,2
10724,3,1,1,1192,4,1,7,1,0,5,...,148,12,30,96,16,10,0,96,70,3
10725,3,1,1,1192,4,1,7,1,0,5,...,148,16,60,93,16,20,0,96,56,3
10726,3,1,1,1192,4,1,7,1,0,5,...,178,16,30,96,17,30,0,96,91,2


Para los modelos con pesos compartidos, es recomendable utilizar el mismo modelo múltiples veces en la función `forward` de la red (3 veces en este caso, una por modo) y calcular la función de pérdida en base a eso. Desde el punto de vista de los datos, es recomendable manipular el `DataFrame` en el `DataLoader`, con el fin de descomponer cada fila en 3 vectores, uno por cada modo. También es posible transformar completamente el `DataFrame` con la función `wide_to_long` de Pandas, de modo de realizar la descomposición completa antes del procesamiento, pero es preferible utilizar el primer esquema por su simplicidad.

In [4]:
#Modelo con pesos compartidos (Hay que calcular función de perdida)
#DataLoader para manipular los datos


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import tqdm





##Modelos puros


###Modelo 1 capa, clasificador con pesos únicos

In [5]:
#x_modos = ['TRAIN_AV', 'TRAIN_TT', 'TRAIN_CO', 'TRAIN_HE', 'CAR_AV', 'CAR_TT', 'CAR_CO', 'SM_TT', 'SM_CO', 'SM_HE', 'SM_SEATS']
#x = ['GROUP', 'SURVEY', 'PURPOSE', 'TICKET','AGE', 'MALE', 'INCOME', 'ORIGIN', 'DEST']
output = ['CHOICE']

x = ['TRAIN_TT', 'TRAIN_CO', 'TRAIN_HE', 'CAR_TT', 'CAR_CO', 'SM_TT', 'SM_CO', 'SM_HE', 
'SM_SEATS','GROUP', 'SURVEY', 'PURPOSE', 'TICKET','AGE', 'MALE', 'INCOME', 'ORIGIN', 'DEST', 'WHO', 'LUGGAGE',
]

#Podriamos aplicar labelcode a las variables categoricas
#features que no se si meter o no 
##SP, first, ga, sm_av, train_av

#Podriamos separar y a las features categoricas aplicar un labelenconder
#for feature in x:
  #df[feature] = LabelEncoder().fit_transform(df[feature])

print(len(x))

20


####Preparar set de datos

In [6]:
class SwissDataset(Dataset):
  def __init__(self, x, y):

    self.n_samples = x.shape[0]
    #Normalizando las variables 
    xnorm = (x - x.mean())/x.std()
    self.x = xnorm.astype(np.float32).values
    self.y = y.astype(np.float32).values


  def __len__(self):
    return self.n_samples


  def __getitem__(self, index):
    return [self.x[index], self.y[index]]


   




In [7]:
#separar set en test y entrenamiento

training, test = train_test_split(df, test_size = 0.2)
training_dataset = SwissDataset(training[x], training[output])
test_dataset = SwissDataset(test[x], test[output])
print(training_dataset.y)

[[2.]
 [2.]
 [3.]
 ...
 [3.]
 [2.]
 [2.]]


####Creación modelo

In [8]:
class SwissMLP(nn.Module):
  def __init__(self,num_input_x, hidden):
    super().__init__()

    self.c1 = nn.Linear(num_input_x , hidden) 
    self.dropout = nn.Dropout(.4)
    self.output = nn.Linear(hidden, 3)  #son 3 modos
    print(self.output)


  def forward(self, x):
    #juntamos las features numericas y categoricas 
    x = self.c1(x)
    x = F.relu(x)
    x = self.dropout(x)
    x = self.output(x)
    y = F.softmax(x)
    return y

   
    

####Instanciar modelo


In [9]:
#Utilizar gpu para correr modelo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
modelo = SwissMLP(num_input_x=  20, hidden = 256).to(device)

Linear(in_features=256, out_features=3, bias=True)


In [10]:
dataloader = DataLoader(training_dataset, batch_size=64, shuffle=True, num_workers=1)

#### Entrenamiento

In [11]:
total_epochs = 1500
learning_rate = 0.01

funcion_perdida = nn.CrossEntropyLoss() #usando la función de perdida usada en el paper
optimizador = torch.optim.Adam(modelo.parameters(), lr= learning_rate)

t_epochs = tqdm.notebook.tqdm(range(total_epochs), unit="epoch")
for epoch in t_epochs:
  t_epochs.set_description(f"Epoch {epoch+1}")


  perdida_total = 0
  for x, y in dataloader: 

    #Cargar los datos en la gpu
    x = x.to(device)
    y = y.to(device)
    y_ = modelo(x)
    print(y_)
    perdida = funcion_perdida(y, y_)#Aqui tenemos que instanciar la funcion de perdida
    optimizador.zero_grad()
    perdida.backward()
    optimizador.step()
    perdida_total += perdida.item()* x.size(0)
  t_epochs.set_postfix(loss = perdida_total/len(training))



###Instanciar modelo


   


  0%|          | 0/1500 [00:00<?, ?epoch/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


tensor([[0.3080, 0.3913, 0.3007],
        [0.2644, 0.3742, 0.3614],
        [0.3445, 0.4017, 0.2538],
        [0.2299, 0.3865, 0.3836],
        [0.3418, 0.2606, 0.3976],
        [0.3766, 0.2937, 0.3297],
        [0.2287, 0.3743, 0.3970],
        [0.3783, 0.3027, 0.3190],
        [0.4266, 0.2783, 0.2950],
        [0.3162, 0.4486, 0.2351],
        [0.4075, 0.3934, 0.1991],
        [0.2878, 0.4065, 0.3058],
        [0.3277, 0.2937, 0.3786],
        [0.2730, 0.3662, 0.3608],
        [0.2977, 0.2814, 0.4209],
        [0.2760, 0.3596, 0.3643],
        [0.2854, 0.3236, 0.3910],
        [0.3828, 0.1922, 0.4249],
        [0.2737, 0.4487, 0.2776],
        [0.1798, 0.2656, 0.5547],
        [0.2801, 0.3437, 0.3761],
        [0.3307, 0.3858, 0.2834],
        [0.2647, 0.3476, 0.3878],
        [0.2262, 0.3368, 0.4370],
        [0.3122, 0.2044, 0.4834],
        [0.3499, 0.3836, 0.2665],
        [0.2870, 0.3945, 0.3185],
        [0.3172, 0.2381, 0.4446],
        [0.2627, 0.3221, 0.4151],
        [0.317

RuntimeError: ignored

####Test

In [ ]:
#Por cada batch se hace un forward

modelo.eval()
#cargas datos
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=True, num_workers=1)
for x, y in test_dataloader: 
  x = x.to(device)
  y = y.to(device)
  y_ = modelo(x)
  print(y_)
  perdida = funcion_perdida(y_, y)

  perdida_total += perdida.item()*x.size(0)

avg_loss = perdida_total/len(test)
print(f"Avg. Loss = {avg_loss:e}")


###Modelo 2 capas, clasificador con pesos únicos


####Preparar set de datos

####Creación modelo

#### Instanciar modelo

####Entrenamiento

####Test

###Modelo 1 capa, clasificador con pesos compartidos


#### Preparacion set de datos

####Creación de modelo

#### Instanciar modelo

#### Entrenamiento

#### Test

###Modelo 2 capas, clasificador con pesos compartidos

#### Preparación set de datos

#### Creación de modelo

#### Instanciar de modelo

#### Entrenamiento

#### Test

### Análisis

##Modelo con embeddings

###Modelo 1 capa, clasificador con pesos únicos

###Modelo 2 capas, clasificador con pesos únicos

###Modelo 1 capa, clasificador con pesos compartidos

###Modelo 2 capas, clasificador con pesos compartidos
